# Все действия были проделаны в google colaboratory на бесплатном GPU
  
<b> Скачивание файлов с google drive из папки DMIA лежащей в корневой папке, их перемещение и установка gensim
  https://drive.google.com/file/d/1fZX_Yu0iD9gAn4lug6m29Fk0-SjF82QK/view?usp=sharing - dataset
  http://rusvectores.org/static/models/rusvectores4/fasttext/araneum_none_fasttextskipgram_300_5_2018.tgz - fasttext
  

In [ ]:
from google.colab import auth
from googleapiclient.discovery import build
import io , requests, os
import sys
auth.authenticate_user()
from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

def get_parent_folder(folder_name):
  page_token = None
  folder_array = []
  query = "name='%s' and mimeType='application/vnd.google-apps.folder'" % folder_name
  while True:
      response = drive_service.files().list(q=query,
                                          spaces='drive',
                                          fields='nextPageToken,files(id, name)',
                                          pageToken=page_token).execute()
      for file in response.get('files', []):
          # Process change
          #print (file.get('name'), file.get('id'))
          folder_array.append({"name" : file.get('name'), "id" : file.get('id')})
      page_token = response.get('nextPageToken', None)
      if page_token is None:
          break
  return folder_array

def get_files_from_parent(parent_id):
  page_token = None
  folder_array = dict()
  query = "'%s' in parents" % parent_id
  while True:
      response = drive_service.files().list(q=query,
                                          spaces='drive',
                                          fields='nextPageToken, files(id, name)',
                                          pageToken=page_token).execute()
      for file in response.get('files', []):
          # Process change
          #print (file.get('name'), file.get('id'))
          folder_array.update({file.get('name'):file.get('id')})
      page_token = response.get('nextPageToken', None)
      if page_token is None:
          break
  return folder_array

def get_file_buffer(file_id, verbose=0):
  from googleapiclient.http import MediaIoBaseDownload
  request = drive_service.files().get_media(fileId=file_id)
  downloaded = io.BytesIO()
  downloader = MediaIoBaseDownload(downloaded, request)
  done = False
  while done is False:
    # _ is a placeholder for a progress object that we ignore.
    # (Our file is small, so we skip reporting progress.)
    progress, done = downloader.next_chunk()
    if verbose:
      sys.stdout.flush()
      sys.stdout.write('\r')
      percentage_done = progress.resumable_progress * 100/progress.total_size
      sys.stdout.write("[%-100s] %d%%" % ('='*int(percentage_done), int(percentage_done)))
  downloaded.seek(0)
  return downloaded

parent_folder = get_parent_folder('DMIA')
print(parent_folder)
parent_folder[0]["id"]
input_file_meta = get_files_from_parent(parent_folder[0]["id"])
print(input_file_meta)
for file, id in input_file_meta.items():
  downloaded = get_file_buffer(id, verbose=1)
  dest_file = os.path.join('.', file)
  print("processing %s data" % file)
  with open(dest_file, "wb") as out:
    out.write(downloaded.read())
    print("Done %s" % dest_file)

!mkdir data
!mv araneum_none_fasttextskipgram_300_5_2018.model data/araneum_none_fasttextskipgram_300_5_2018.model
!mv train.tsv data/train.tsv
!mv public.tsv data/public.tsv
!mv araneum_none_fasttextskipgram_300_5_2018.model.syn0_vocab_lockf.npy data/araneum_none_fasttextskipgram_300_5_2018.model.syn0_vocab_lockf.npy
!mv araneum_none_fasttextskipgram_300_5_2018.model.syn1neg.npy data/araneum_none_fasttextskipgram_300_5_2018.model.syn1neg.npy
!mv araneum_none_fasttextskipgram_300_5_2018.model.wv.syn0.npy data/araneum_none_fasttextskipgram_300_5_2018.model.wv.syn0.npy
!mv araneum_none_fasttextskipgram_300_5_2018.model.wv.syn0_vocab.npy data/araneum_none_fasttextskipgram_300_5_2018.model.wv.syn0_vocab.npy

!pip install gensim

<b> Описание данных и задачи </b>

Каждый эпизод состоит из двух частей – контекста (Context) и финальной реплики (Reply). Например,

- context_2: Персонаж A говорит реплику 
- context_1: Персонаж B отвечает на нее 
- context_0: Персонаж А произносит вторую реплику 
- reply: Персонаж B отвечает на вторую реплику 

Контекстная часть может состоять из трех реплик (как в примере) – в 50% случаев, двух – в 25%, и одного – в оставшихся 25% случаев. Финальная реплика (Reply) всегда завершает любой эпизод, то есть следует за контекстом (Context). Задача участников – найти наиболее подходящую и интересную реплику для данного контекста среди предложенных кандидатов (числом до 6), случайно выбранных из топа кандидатов, возвращенных бейзлайном высокого качества, натренированным командой Алисы (который, в свою очередь, отобрал кандидатов среди всех возможных реплик OpenSubtitles).

Все реплики-кандидаты размечены асессорами на сервисе Яндекс.Толока с помощью следующей инструкции для разметки:

- Good (2): реплика уместна (имеет смысл для данного контекста) и интересна (нетривиальна, специфична именно для данного контекста, мотивирует продолжать разговор)
- Neutral (1): реплика уместна (имеет смысл для данного контекста), но не интересна (тривиальна, не специфична для данного контекста и скорее подталкивает пользователя закончить разговор)
- Bad (0): реплика не имеет никакого смысла в данном контексте

Каждая метка в тренировочной части датасета (и только в ней), сопровождается также уверенностью (confidence) – числом в интервале от 0 до 1 – которое показывает насколько уверенными в своей разметке были асессоры с Толоки, совместно предложившие данную метку. Мы хотим обратить особое внимание участников на эту информацию, она может быть очень полезна при обучении их моделей.

- context_id – идентификатор эпизода
- context_2,context_1,context_0 – текст реплик, предшествующих финальной (может состоять из трех частей)
- reply_id – идентификатор реплики-кандидата
- reply – текст реплики-кандидата
- label – метка реплики-кандидата (good, neutral или bad)
- confidence - уверенность в метке реплики-кандидата (число от 0 до 1)

In [ ]:
import pandas as pd
import numpy as np

from keras.models import Sequential, Model
from keras.layers import *
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import optimizers

import matplotlib.pyplot as plt
import seaborn as sns
import gensim
ft = gensim.models.FastText.load('data/araneum_none_fasttextskipgram_300_5_2018.model')

import re

from sklearn.metrics import make_scorer, accuracy_score
%matplotlib inline

<b> Метрика для оценки качества. Выбрана исходя из задачи ранжирования.

In [ ]:
def DCG(label): return sum([float(label[i]/np.log2(i+2)) for i in range(len(label))])

def nDCG(label, best_label):
    label, best_label = DCG(label), DCG(best_label)
    if label != 0 and best_label != 0:
        return label/best_label
    else:
        return 0

<b> Загружаем данные, конкатенируем трейн и тест

In [ ]:
df_train = pd.read_csv("data/train.tsv", sep='\t', quotechar=' ', header = None)
df_train.columns = ['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply', 'label', 'confidence']

df_public = pd.read_csv("data/public.tsv", sep='\t', quotechar=' ', header = None)
df_public.columns = ['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply']

df_data = pd.concat((df_train, df_public)).reset_index(drop=True)

<b>Заменяем лейблы и заполняем пустые значения confidence на 1 (Почему так рассмотрим далее), остальные на -, меняем порядок столбцов

In [ ]:
df_data.label = df_data.label.map({'good': 2, 'neutral': 1, 'bad': 0})
df_data.confidence.fillna(value=1, inplace=True)
df_data.fillna(value='-', inplace=True)
df_data = df_data[['context_id', 'context_2', 'context_1', 'context_0', 'reply_id', 'reply', 'label', 'confidence']]

<b> Выделяем из общего датафрейма трейн и тест (для удобства дальнейшей визуализации)

In [ ]:
train_tres = len(df_train)
df_train = df_data.loc[:train_tres-1]
df_public = df_data.loc[train_tres:]

df_train.confidence = df_train.confidence.astype(float)
df_train.label = df_train.label.astype(int)

# Часть визуализации

<b> Смотрим на распределения confidence и лейблов по трейну

In [ ]:
fig, ax = plt.subplots(nrows=1, ncols=2, figsize = (15,3))
sns.distplot(df_train['confidence'], bins=100, kde=False, ax=ax[0], color=['b'])
sns.countplot(x='label', data=df_train, ax=ax[1]);

<b> Смотрим на распределения кол-ва слов в context и reply в трейн и тесте (слова выделялись пробелами)

In [ ]:
%%time
fig, ax = plt.subplots(nrows=2, ncols=4, figsize=(18,10))

for i in range(3):
    sns.distplot(df_train[f'context_{2-i}'].apply(lambda x: len(str(x).split(' '))), bins=100, kde=False, ax=ax[0,i])
    ax[0,i].set_title('train')
sns.distplot(df_train['reply'].apply(lambda x: len(str(x).split(' '))), bins=100, kde=False, ax=ax[0,i+1])
ax[0,i+1].set_title('train')

for i in range(3):
    sns.distplot(df_public[f'context_{2-i}'].apply(lambda x: len(str(x).split(' '))), bins=100, kde=False, ax=ax[1,i])
    ax[1,i].set_title('public')
sns.distplot(df_public['reply'].apply(lambda x: len(str(x).split(' '))), bins=100, kde=False, ax=ax[1,i+1])
ax[1,i+1].set_title('public')

<b>Как видно из картинок в тесте и трейне кол-ва слов распределены примерно одинаково

<b>Далее смотрим на длину всего диалога в тесте и трейне

In [ ]:
%%time
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(18,3))

sns.distplot(df_train[['context_2', 'context_1', 'context_0', 'reply']].apply(lambda x : 
                                                                              len((" ".join(x.astype(str))).split(' ')), 
                                                                              axis = 1),
            bins=100, kde=False, ax=ax[0])
ax[0].set_title('train dialog words length')

sns.distplot(df_public[['context_2', 'context_1', 'context_0', 'reply']].apply(lambda x : 
                                                                              len((" ".join(x.astype(str))).split(' ')), 
                                                                              axis = 1),
            bins=100, kde=False, ax=ax[1])
ax[1].set_title('public dialog words length')

<b>Как и ожидалось распределения одинаковы

<b>Теперь подсчитываем из скольки контекстов состоит диалог и смотрим распределения

In [ ]:
%%time
fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(18,3))

df_train['dialog_len'] = df_train[['context_2', 'context_1', 'context_0']].apply(lambda x: 3-sum(x=='-'), axis=1)
sns.countplot(x='dialog_len', data=df_train, ax=ax[0]);
ax[0].set_title('train dialog length')

df_public['dialog_len'] = df_public[['context_2', 'context_1', 'context_0']].apply(lambda x: 3-sum(x=='-'), axis=1)
sns.countplot(x='dialog_len', data=df_public, ax=ax[1]);
ax[1].set_title('public dialog length')

<b>Опять таки мы видим, что распределения в трейне и тесте одинаковы

<b>Далее проверяем граничные значения (Например, одно слово в context или reply), а именно: 
- распределение confidence и лейблов для длины диалога = 1
- распределение confidence и лейблов для длины reply = 1
- распределение confidence и лейблов для длины reply = 2
- распределение confidence и лейблов для длины reply >= 2
- распределение confidence и лейблов для длины reply = 1 и длины диалога = 1

In [ ]:
%%time
fig, ax = plt.subplots(nrows=1, ncols=2, figsize = (15,2))
sns.distplot(df_train[df_train['dialog_len'] == 1].confidence, bins=100, kde=False, ax=ax[0])
sns.countplot(x='label', data=df_train[df_train['dialog_len'] == 1], ax=ax[1])
fig.suptitle('train dialog length==1 statistics')

fig, ax = plt.subplots(nrows=1, ncols=2, figsize = (15,2))
sns.distplot(df_train[df_train.reply.apply(lambda x: len(x.split()) == 1)].confidence, bins=100, kde=False, ax=ax[0])
sns.countplot(x='label', data=df_train[df_train.reply.apply(lambda x: len(x.split()) == 1)], ax=ax[1])
fig.suptitle('train reply length==1 statistics')

fig, ax = plt.subplots(nrows=1, ncols=2, figsize = (15,2))
sns.distplot(df_train[df_train.reply.apply(lambda x: len(x.split()) == 2)].confidence, bins=100, kde=False, ax=ax[0])
sns.countplot(x='label', data=df_train[df_train.reply.apply(lambda x: len(x.split()) == 2)], ax=ax[1])
fig.suptitle('train reply length==2 statistics')

fig, ax = plt.subplots(nrows=1, ncols=2, figsize = (15,2))
sns.distplot(df_train[df_train.reply.apply(lambda x: len(x.split()) >= 2)].confidence, bins=100, kde=False, ax=ax[0])
sns.countplot(x='label', data=df_train[df_train.reply.apply(lambda x: len(x.split()) > 2)], ax=ax[1])
fig.suptitle('train reply length>=2 statistics')

fig, ax = plt.subplots(nrows=1, ncols=2, figsize = (15,2))
sns.distplot(df_train[df_train.reply.apply(lambda x: len(x.split()) == 1) & (df_train['dialog_len'] == 1)].confidence, bins=100, kde=False, ax=ax[0])
sns.countplot(x='label', data=df_train[df_train.reply.apply(lambda x: len(x.split()) == 1) & (df_train['dialog_len'] == 1)], ax=ax[1])
fig.suptitle('train reply length==1 & dialog_len==1 statistics')

<b>Как мы видим при длине диалога 1 и длине reply 1 у нас почти в большинстве случаев лейбл 0

<b>Дальше попытаемся как-то использовать confidence. В данном случае confidence для других лейблов считался как (1-confidence)/2 и для этой формулы confidence для тестовой части мы взяли как 1. В этом случае confidence для лейблов для тестовой части будет 0. Можно было избежать мороки с confidence для тестовой части 1 просто сделав срез для трейн части, но суть от этого не изменится. <br>

В итоге мы расширили фичу confidence и будем их использовать как лейбл

In [ ]:
%%time
df_data['inv_confidence_div2'] = (1 - df_data.confidence)/2

df_data['label0_confidence'] = 0
df_data['label1_confidence'] = 0
df_data['label2_confidence'] = 0

df_data['label0_confidence'] = df_data[['label0_confidence', 'label', 'inv_confidence_div2', 'confidence']].apply(lambda x: x.confidence if x.label==0 else x.inv_confidence_div2 ,axis=1)
df_data['label1_confidence'] = df_data[['label1_confidence', 'label', 'inv_confidence_div2', 'confidence']].apply(lambda x: x.confidence if x.label==1 else x.inv_confidence_div2 ,axis=1)
df_data['label2_confidence'] = df_data[['label2_confidence', 'label', 'inv_confidence_div2', 'confidence']].apply(lambda x: x.confidence if x.label==2 else x.inv_confidence_div2 ,axis=1)

<b>Теперь нужно избавиться от всех символов кроме русских (в датасете могут присутствовать не русские символы и на самом деле так и есть) , так как наш эмбеддинг над словами будет из fasttext. Сделаем это с помощью регулярных выражений

In [ ]:
%%time
reg = re.compile('[^а-яА-Я]')
df_data['context_2'] = df_data['context_2'].apply(lambda x: reg.sub(r' ', x))
df_data['context_1'] = df_data['context_1'].apply(lambda x: reg.sub(r' ', x))
df_data['context_0'] = df_data['context_0'].apply(lambda x: reg.sub(r' ', x))
df_data['reply'] = df_data['reply'].apply(lambda x: reg.sub(r' ', x))

<b>Теперь возьмем слова с длиной 3 символа и более (потому что наш эмбеддинг обучался на 3-5 грамах) и обрежем наши context и reply. Для того, чтобы понять на сколько обрезать нужно смотреть на наши графики с распределением кол-ва слов. В итоге имеем: context2 - 10 слов, context1 - 15 слов, context0 - 20 слов, reply - 15 слов.

In [ ]:
%%time
temp = df_data['context_2'].apply(lambda x: x.split(' '))
df_data['context_2_cutted'] = temp.apply( lambda x: list(filter(lambda y : len(y) >= 3, x))[:10] )

temp = df_data['context_1'].apply(lambda x: x.split(' '))
df_data['context_1_cutted'] = temp.apply(lambda x: list(filter(lambda y : len(y) >= 3, x))[:15] )

temp = df_data['context_0'].apply(lambda x: x.split(' '))
df_data['context_0_cutted'] = temp.apply(lambda x: list(filter(lambda y : len(y) >= 3, x))[:20] )

temp = df_data['reply'].apply(lambda x: x.split(' '))
df_data['reply_cutted'] = temp.apply(lambda x: list(filter(lambda y : len(y) >= 3, x))[:15] )

<b>Найдем и удалим из данных строки, где после наших махинаций длина reply или context0 равны 0

In [ ]:
%%time
data_zero_reply_loc = df_data['reply_cutted'][df_data['reply_cutted'].apply(lambda x: len(x)==0)].index.values
data_zero_context_0_loc = df_data['context_0_cutted'][df_data['context_0_cutted'].apply(lambda x: len(x)==0)].index.values

data_noise = list(set(data_zero_reply_loc) | set(data_zero_context_0_loc))
df_data_cleared = df_data.drop(labels=data_noise)
print(len(data_noise))

In [ ]:
df_data_cleared.sample()

# Часть кросс валидации
<b>Теперь разберемся с кроссвалидацией и моделью. <br>
Модель выбрана нейросетевая (хотя она тут не очень подходит, так как данных для нейронной сети (глубокой) очень мало, порядка 70 тыс.) <br>
Валидационная выборка это будут последние 20% теста. Данный тип валидации подходит, так как здесь нет временной зависимости или какой-либо еще не тривиальной.

Модель: 2 входа context и reply, далее embedding на fasttext, CNN для context и для reply. Потом идет конкатенация двух CNN, потом Dense слой и слой softmax. loss - categorical_crossentropy, оптимизатор - AMSgrad.

In [ ]:
df_train = df_data_cleared.loc[:train_tres]
CV_COEF = 0.8
cv_train_tres = int(len(df_train) * CV_COEF)
cv_train_y = df_train[['label0_confidence', 'label1_confidence', 'label2_confidence']][:cv_train_tres].values
cv_val_y = df_train[['label0_confidence', 'label1_confidence', 'label2_confidence']][cv_train_tres:].values

<b>Создадим данные для входа в нейросеть (конатенация диалога без reply)

In [ ]:
cv_train_context = df_train['context_2_cutted'] + df_train['context_1_cutted'] + df_train['context_0_cutted']
cv_train_reply = df_train['reply_cutted']

<b>Определим максимальную длину диалога для каждого входа, размер эмбеддинга (задается самой моделью fasttext-а) и кол-во токенов

In [ ]:
MAX_NB_WORDS = 100000
MAX_SEQUENCE_LENGTH_CONTEXT = 45
MAX_SEQUENCE_LENGTH_REPLY = 15
EMBEDDING_DIM = 300

<b>Создадим токены, приведем входные данные к одной длине и разобьем данные на трейн и валидацию

In [ ]:
%%time

tokenizer_context = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer_context.fit_on_texts(cv_train_context)
sequences_context = tokenizer_context.texts_to_sequences(cv_train_context)
word_index_context = tokenizer_context.word_index
data_context = pad_sequences(sequences_context, maxlen=MAX_SEQUENCE_LENGTH_CONTEXT)

X_train_context = data_context[:cv_train_tres]
X_val_context = data_context[cv_train_tres:]

#################################################################################

tokenizer_reply = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer_reply.fit_on_texts(cv_train_reply)
sequences_reply = tokenizer_reply.texts_to_sequences(cv_train_reply)
word_index_reply = tokenizer_reply.word_index
data_reply = pad_sequences(sequences_reply, maxlen=MAX_SEQUENCE_LENGTH_REPLY)

X_train_reply = data_reply[:cv_train_tres]
X_val_reply = data_reply[cv_train_tres:]

<b>Создадим эмбеддинги для слов

In [ ]:
%%time

input_context = Input(shape=(MAX_SEQUENCE_LENGTH_CONTEXT, ))
embedding_matrix_context = np.zeros((len(word_index_context) + 1, EMBEDDING_DIM))
for word, i in word_index_context.items():
    embedding_vector = ft.wv[word]
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_context[i] = embedding_vector
word_embed_context = Embedding(len(word_index_context) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix_context],
                            input_length=MAX_SEQUENCE_LENGTH_CONTEXT,
                            trainable=False)(input_context)

##############################################################################

input_reply = Input(shape=(MAX_SEQUENCE_LENGTH_REPLY, ))
embedding_matrix_reply = np.zeros((len(word_index_reply) + 1, EMBEDDING_DIM))
for word, i in word_index_reply.items():
    embedding_vector = ft.wv[word]
    if embedding_vector is not None:
        embedding_matrix_reply[i] = embedding_vector
word_embed_reply = Embedding(len(word_index_reply) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix_reply],
                            input_length=MAX_SEQUENCE_LENGTH_REPLY,
                            trainable=False)(input_reply)

<b>Определим модель и начнем обучение на 10 эпохах с батчем в 64. <br>
  Парамерты и архитектура модели подбирались на основе валидации путем проб и ошибок.

In [ ]:
BATCH_SIZE = 64

context_conv1 = Conv1D(16, 5, padding='same')(word_embed_context)
context_max_pool1 = MaxPooling1D(pool_size=2)(context_conv1)
context_conv2 = Conv1D(32, 3, padding='same')(context_max_pool1)
context_max_pool2 = MaxPooling1D(pool_size=2)(context_conv2)
context_conv3 = Conv1D(64, 3, padding='same')(context_max_pool2)
context_max_pool3 = MaxPooling1D(pool_size=2)(context_conv3)
context_flatten = Flatten()(context_max_pool3)

reply_conv1 = Conv1D(32, 5, padding='same')(word_embed_reply)
reply_max_pool1 = MaxPooling1D(pool_size=2)(reply_conv1)
reply_conv2 = Conv1D(64, 3, padding='same')(reply_max_pool1)
reply_max_pool2 = MaxPooling1D(pool_size=2)(reply_conv2)
reply_flatten = Flatten()(reply_max_pool2)

merged = concatenate([context_flatten, reply_flatten])
drop_0 = Dropout(0.3)(merged)
dense_0 = Dense(256, activation='sigmoid')(drop_0)
drop_1 = Dropout(0.3)(dense_0)
dense_out = Dense(3, activation='softmax')(drop_1)

model = Model(inputs=[input_context, input_reply], outputs=[dense_out])
model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

hist = model.fit([X_train_context, X_train_reply], cv_train_y, batch_size=BATCH_SIZE, epochs=10, validation_data=([X_val_context, X_val_reply], cv_val_y))
plt.figure()
plt.title('acc')
plt.plot(hist.history['val_acc'], label='val')
plt.plot(hist.history['acc'], label='train')
plt.legend()
plt.figure()
plt.title('loss')
plt.plot(hist.history['val_loss'], label='val')
plt.plot(hist.history['loss'], label='train')
plt.legend()

<b> Теперь обучим модель на 6 эпохах (выбор был сделан исходя из графика лосса на валидации - минимальная точка на графике)

In [ ]:
context_conv1 = Conv1D(16, 5, padding='same')(word_embed_context)
context_max_pool1 = MaxPooling1D(pool_size=2)(context_conv1)
context_conv2 = Conv1D(32, 3, padding='same')(context_max_pool1)
context_max_pool2 = MaxPooling1D(pool_size=2)(context_conv2)
context_conv3 = Conv1D(64, 3, padding='same')(context_max_pool2)
context_max_pool3 = MaxPooling1D(pool_size=2)(context_conv3)
context_flatten = Flatten()(context_max_pool3)

reply_conv1 = Conv1D(32, 5, padding='same')(word_embed_reply)
reply_max_pool1 = MaxPooling1D(pool_size=2)(reply_conv1)
reply_conv2 = Conv1D(64, 3, padding='same')(reply_max_pool1)
reply_max_pool2 = MaxPooling1D(pool_size=2)(reply_conv2)
reply_flatten = Flatten()(reply_max_pool2)

merged = concatenate([context_flatten, reply_flatten])
drop_0 = Dropout(0.3)(merged)
dense_0 = Dense(256, activation='sigmoid')(drop_0)
drop_1 = Dropout(0.3)(dense_0)
dense_out = Dense(3, activation='softmax')(drop_1)

model = Model(inputs=[input_context, input_reply], outputs=[dense_out])
model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit([X_train_context, X_train_reply], cv_train_y, batch_size=BATCH_SIZE, epochs=6, validation_data=([X_val_context, X_val_reply], cv_val_y))

<b> Делаем предикт для валидационной выборке и считаем скор

In [ ]:
y_pred = model.predict([X_val_context, X_val_reply])
y_pred_classes = np.argmax(y_pred, axis=1)
cv_val_classes = np.argmax(cv_val_y, axis=1)
nDCG(cv_val_classes, y_pred_classes)

# Часть обучения


<b> Отделим тест и трейн

In [ ]:
%%time
train_inds = df_data[df_data.label != '-'].index.values
test_inds = df_data[df_data.label == '-'].index.values
y_train = df_data[df_data.label != '-'][['label0_confidence', 'label1_confidence', 'label2_confidence']].values

context = df_data['context_2_cutted'] + df_data['context_1_cutted'] + df_data['context_0_cutted']
reply = df_data['reply_cutted']

<b>Сделаем те же процедуры для эмбединга, какие делали для кроссвалидации 


In [ ]:
%%time

tokenizer_context = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer_context.fit_on_texts(context)
sequences_context = tokenizer_context.texts_to_sequences(context)
word_index_context = tokenizer_context.word_index
data_context = pad_sequences(sequences_context, maxlen=MAX_SEQUENCE_LENGTH_CONTEXT)

train_context = data_context[train_inds]
test_context = data_context[test_inds]

#################################################################################

tokenizer_reply = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer_reply.fit_on_texts(reply)
sequences_reply = tokenizer_reply.texts_to_sequences(reply)
word_index_reply = tokenizer_reply.word_index
data_reply = pad_sequences(sequences_reply, maxlen=MAX_SEQUENCE_LENGTH_REPLY)

train_reply = data_reply[train_inds]
test_reply = data_reply[test_inds]

In [ ]:
%%time

input_context = Input(shape=(MAX_SEQUENCE_LENGTH_CONTEXT, ))
embedding_matrix_context = np.zeros((len(word_index_context) + 1, EMBEDDING_DIM))
for word, i in word_index_context.items():
    embedding_vector = ft.wv[word]
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_context[i] = embedding_vector
word_embed_context = Embedding(len(word_index_context) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix_context],
                            input_length=MAX_SEQUENCE_LENGTH_CONTEXT,
                            trainable=False)(input_context)

##############################################################################

input_reply = Input(shape=(MAX_SEQUENCE_LENGTH_REPLY, ))
embedding_matrix_reply = np.zeros((len(word_index_reply) + 1, EMBEDDING_DIM))
for word, i in word_index_reply.items():
    embedding_vector = ft.wv[word]
    if embedding_vector is not None:
        embedding_matrix_reply[i] = embedding_vector
word_embed_reply = Embedding(len(word_index_reply) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix_reply],
                            input_length=MAX_SEQUENCE_LENGTH_REPLY,
                            trainable=False)(input_reply)

<b> Обучим сеть на всем тренировочном датасете с количеством эпох в 6



In [ ]:
BATCH_SIZE = 64

context_conv1 = Conv1D(16, 5, padding='same')(word_embed_context)
context_max_pool1 = MaxPooling1D(pool_size=2)(context_conv1)
context_conv2 = Conv1D(32, 3, padding='same')(context_max_pool1)
context_max_pool2 = MaxPooling1D(pool_size=2)(context_conv2)
context_conv3 = Conv1D(64, 3, padding='same')(context_max_pool2)
context_max_pool3 = MaxPooling1D(pool_size=2)(context_conv3)
context_flatten = Flatten()(context_max_pool3)

reply_conv1 = Conv1D(32, 3, padding='same')(word_embed_reply)
reply_max_pool1 = MaxPooling1D(pool_size=2)(reply_conv1)
reply_conv2 = Conv1D(64, 3, padding='same')(reply_max_pool1)
reply_max_pool2 = MaxPooling1D(pool_size=2)(reply_conv2)
reply_flatten = Flatten()(reply_max_pool2)

merged = concatenate([context_flatten, reply_flatten])
drop_0 = Dropout(0.3)(merged)
dense_0 = Dense(256, activation='sigmoid')(drop_0)
drop_1 = Dropout(0.3)(dense_0)
dense_out = Dense(3, activation='softmax')(drop_1)

model = Model(inputs=[input_context, input_reply], outputs=[dense_out])
model.compile(optimizer=optimizers.Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

print(model.summary())

model.fit([train_context, train_reply], y_train, batch_size=BATCH_SIZE, epochs=6)

<b> Сделаем предикт по тесту и сохраним в csv файле

In [ ]:
y_pred = model.predict([test_context, test_reply])
y_pred_classes = np.argmax(y_pred, axis=1)
y_pred_prob = np.max(y_pred, axis=1)

In [ ]:
df_public['label'] = y_pred_classes
df_public['confidence'] = y_pred_prob
df_public.sort_values(by=['context_id', 'confidence'], ascending=False)[['context_id', 'reply_id']].to_csv('subm.csv', encoding='utf-8', sep=' ', index=False)

<b> Скачаем csv файл с колаба

In [ ]:
from google.colab import files
files.download('subm.csv')

<b> Скор 83919, что соответствует ожиданиям исходя скору на валидации